# Naive Bayes

In [1]:
import numpy as np
import pandas as pd
import sklearn
import nltk

In [2]:
data = pd.read_csv("combined_data.csv")
data

,sentiment,Text
0,sadness,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,happy,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
61454,fear,Melissa stared at her friend in dism
61455,happy,Successive state elections have seen the gover...
61456,fear,Vincent was irritated but not dismay
61457,happy,Kendall-Hume turned back to face the dismayed ...


In [3]:
data["sentiment"].value_counts()

fear        16241
happy       13508
sadness      9796
neutral      8960
love         4720
anger        4069
surprise     2639
relief       1526
Name: sentiment, dtype: int64

In [4]:
# Removing punctuation, URL, and tags
import re
data['Text'] = data['Text'].apply(lambda x: re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)",'',x))


In [5]:
data.dropna(inplace=True)

In [6]:
print(data.shape)
data.head()
print(data.tail())

(61459, 2)
      sentiment                                               Text
61454      fear               Melissa stared at her friend in dism
61455     happy  Successive state elections have seen the gover...
61456      fear               Vincent was irritated but not dismay
61457     happy  KendallHume turned back to face the dismayed coup
61458     happy                     I am dismayed  but not surpris


In [7]:
import snowballstemmer
ss = snowballstemmer.stemmer('english')
def replace(x):
    words = x.split()
    newtext = ''
    for w in words:
        n = ss.stemWord(w)
        newtext += n
        newtext += " "
    return newtext
# stem the text
data['Text'] = data['Text'].apply(lambda x: replace(x))
data.head()

,sentiment,Text
0,sadness,i know i was listenin to bad habit earlier and...
1,sadness,Layin n bed with a headach ughhhhwaitin on you...
2,sadness,Funer ceremonygloomi friday
3,happy,want to hang out with friend SOON
4,neutral,We want to trade with someon who has Houston t...


In [8]:
data['sentiment'].value_counts()

fear        16241
happy       13508
sadness      9796
neutral      8960
love         4720
anger        4069
surprise     2639
relief       1526
Name: sentiment, dtype: int64

In [9]:
#function to build a new non-skewed dataset

#sample with replacement
def stratify(data, N):
    rows = []
    fear = data[data['sentiment'] == 'fear']
    happy = data[data['sentiment'] == 'happy']
    sad = data[data['sentiment'] == 'sadness']
    neutral = data[data['sentiment'] == 'neutral']
    love = data[data['sentiment'] == 'love']
    anger = data[data['sentiment'] == 'anger']
    surprise = data[data['sentiment'] == 'surprise']
    relief = data[data['sentiment'] == 'relief']
    
    for i in range(N):
        #print(fear.loc[np.random.choice(fear.index)])
        rows.append(fear.loc[np.random.choice(fear.index)])
        rows.append(happy.loc[np.random.choice(happy.index)])
        rows.append(sad.loc[np.random.choice(sad.index)])
        rows.append(neutral.loc[np.random.choice(neutral.index)])
        rows.append(love.loc[np.random.choice(love.index)])
        rows.append(anger.loc[np.random.choice(anger.index)])
        rows.append(surprise.loc[np.random.choice(surprise.index)])
        rows.append(relief.loc[np.random.choice(relief.index)])
    sentiments = [x['sentiment'] for x in rows]
    texts = [x['Text'] for x in rows]
    d = {'sentiment': sentiments, 'Text': texts}
    return pd.DataFrame(d)


#### Split Train and Test sets:

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['Text'], 
                                                    data['sentiment'],test_size=0.10, 
                                                    random_state=1)


In [11]:
y_train.value_counts()

fear        14543
happy       12239
sadness      8795
neutral      8040
love         4278
anger        3669
surprise     2374
relief       1375
Name: sentiment, dtype: int64

In [12]:
y_test.value_counts()

fear        1698
happy       1269
sadness     1001
neutral      920
love         442
anger        400
surprise     265
relief       151
Name: sentiment, dtype: int64

#### Stratification (oversampling) of train and test sets
Creation of balanced train and test sets

In [13]:
# join test and train X and y's together to stratify
data_train = pd.concat([X_train,y_train],axis=1)
data_test =  pd.concat([X_test,y_test],axis=1)
train_balanced = stratify(data_train, 8000)
test_balanced  = stratify(data_test, 2000)

In [14]:
train_balanced["sentiment"].value_counts()

neutral     8000
sadness     8000
happy       8000
fear        8000
anger       8000
relief      8000
surprise    8000
love        8000
Name: sentiment, dtype: int64

In [15]:
test_balanced["sentiment"].value_counts()

happy       2000
love        2000
neutral     2000
sadness     2000
fear        2000
anger       2000
relief      2000
surprise    2000
Name: sentiment, dtype: int64

In [16]:
# create X and y vectors for balanced training set
strat_X_train = train_balanced["Text"]
strat_y_train = train_balanced["sentiment"]

In [17]:
# create X and y vectors for balanced test set
strat_X_test = test_balanced["Text"]
strat_y_test = test_balanced["sentiment"]

## Creating a document matrix

#### Training set

In [18]:
# Applying the count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english', max_features = 10000)
# unbalanced train
X_train_counts = count_vect.fit_transform(X_train)
# balanced train
X_train_strat_counts = count_vect.fit_transform(strat_X_train)

In [19]:
# Applying the TFidf vectorizer
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
# unbalanced train
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# balanced train
X_train_strat_tfidf = tfidf_transformer.fit_transform(X_train_strat_counts)

#### Test set

In [20]:
# unbalanced testset
# count vectorizer
X_test_counts = count_vect.transform(X_test)
# Tfidf transformer
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

# balanced testset
# count vectorizer
X_test_strat_counts = count_vect.transform(strat_X_test)
# Tfidf transformer
X_test_strat_tfidf = tfidf_transformer.transform(X_test_strat_counts)

## Multinomial Naive Bayes

In [21]:
from sklearn.naive_bayes import MultinomialNB

#### Training on unbalanced data set

In [22]:
clf1 = MultinomialNB().fit(X_train_tfidf, y_train)

#### Training on a balanced dataset

In [23]:
clf2 = MultinomialNB().fit(X_train_strat_tfidf, strat_y_train)

#### Applying the models

In [24]:
prediction1 = clf1.predict(X_test_tfidf)
prediction2 = clf2.predict(X_test_strat_tfidf)
prediction3 = clf1.predict(X_test_strat_tfidf)
prediction4 = clf2.predict(X_test_tfidf)

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Skewed training and skewed test data:')
print('Accuracy score: {}'.format(accuracy_score(y_test, prediction1)))
print('precision score: {}'.format(precision_score(y_test, prediction1, average='macro')))
print('recall score: {}'.format(recall_score(y_test, prediction1,average='macro')))
print('f1 score: {}'.format(f1_score(y_test, prediction1,average='macro')))

print('Balanced training and balanced test data:')
print('Accuracy score: {}'.format(accuracy_score(strat_y_test, prediction2)))
print('precision score: {}'.format(precision_score(strat_y_test, prediction2, average='macro')))
print('recall score: {}'.format(recall_score(strat_y_test, prediction2,average='macro')))
print('f1 score: {}'.format(f1_score(strat_y_test, prediction2,average='macro')))

print('Skewed training and balanced test data:')
print('Accuracy score: {}'.format(accuracy_score(strat_y_test, prediction3)))
print('precision score: {}'.format(precision_score(strat_y_test, prediction3, average='macro')))
print('recall score: {}'.format(recall_score(strat_y_test, prediction3,average='macro')))
print('f1 score: {}'.format(f1_score(strat_y_test, prediction3,average='macro')))

print('Balanced training and skewed test data:')
print('Accuracy score: {}'.format(accuracy_score(y_test, prediction4)))
print('precision score: {}'.format(precision_score(y_test, prediction4, average='macro')))
print('recall score: {}'.format(recall_score(y_test, prediction4,average='macro')))
print('f1 score: {}'.format(f1_score(y_test, prediction4,average='macro')))

Skewed training and skewed test data:
Accuracy score: 0.19166937845753335
precision score: 0.11800763140649356
recall score: 0.12125000440788507
f1 score: 0.1152046264338105
Balanced training and balanced test data:
Accuracy score: 0.3183125
precision score: 0.32014977529847
recall score: 0.3183125
f1 score: 0.3172290706060352
Skewed training and balanced test data:
Accuracy score: 0.1195625
precision score: 0.10855570433805978
recall score: 0.1195625
f1 score: 0.09058121219336607
Balanced training and skewed test data:
Accuracy score: 0.3042629352424341
precision score: 0.2812125208523259
recall score: 0.32100334631623134
f1 score: 0.2798283970821942


## Generalization on unseen real world data

In [26]:
dfReal = pd.read_csv('realWorldEmotions.csv')

In [27]:
# Removing punctuation, URL, and tags
dfReal['Text'] = dfReal['Text'].apply(lambda x: re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)",'',x))

In [28]:
dfReal.dropna(inplace=True)

In [29]:
# stem the text
dfReal['Text'] = dfReal['Text'].apply(lambda x: replace(x))

In [30]:
# check class imbalance
dfReal['Sentiment'].value_counts()

joy         695
sadness     581
anger       275
fear        224
love        159
surprise     66
Name: Sentiment, dtype: int64

In [31]:
#Change "joy" to "happy" to match categories
for i,s in enumerate(dfReal['Sentiment']):
    if s == 'joy':
        dfReal.loc[i,'Sentiment'] = 'happy'

In [32]:
dfReal["Sentiment"].value_counts()

happy       695
sadness     581
anger       275
fear        224
love        159
surprise     66
Name: Sentiment, dtype: int64

In [33]:
# Conversion to document matrix 
X = dfReal["Text"]
y = dfReal["Sentiment"]
# count vectorizer
X_count = count_vect.transform(X)
# Tfidf transformer
X_tfidf = tfidf_transformer.transform(X_count)

In [34]:
# prediction on unbalanced trained model
prediction_real1 = clf1.predict(X_tfidf)

In [35]:
# prediction on balanced trained model
prediction_real2 = clf2.predict(X_tfidf)

In [36]:
print('Skewed training model tested on real data:')
print('Accuracy score: {}'.format(accuracy_score(y, prediction_real1)))
print('precision score: {}'.format(precision_score(y, prediction_real1, average='macro')))
print('recall score: {}'.format(recall_score(y, prediction_real1,average='macro')))
print('f1 score: {}'.format(f1_score(y, prediction_real1,average='macro')))

print('Balanced training model tested on real data:')
print('Accuracy score: {}'.format(accuracy_score(y, prediction_real2)))
print('precision score: {}'.format(precision_score(y, prediction_real2, average='macro')))
print('recall score: {}'.format(recall_score(y, prediction_real2,average='macro')))
print('f1 score: {}'.format(f1_score(y, prediction_real2,average='macro')))

Skewed training model tested on real data:
Accuracy score: 0.1675
precision score: 0.1280475983580936
recall score: 0.0994087626008256
f1 score: 0.09811740192174975
Balanced training model tested on real data:
Accuracy score: 0.613
precision score: 0.47339887880257336
recall score: 0.44917001200263795
f1 score: 0.43344096454824915


C:\Users\aditi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\aditi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


I noticed that some predictions were categories which weren't in the real dataset (like "neutral"). This is why we see the error above.